<a href="https://colab.research.google.com/github/obdegirmenci/colablend/blob/master/colablend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check Hardware

In [ ]:
!nvidia-smi -q -i 0 | grep "Product Name"

#Connect to Google Drive

Specify your desired blender version and the path to your blend file within google drive.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# Settings

In [ ]:
#@title # **Blender** Configuration { form-width: "35%" }
#@markdown ## *Use the form that has various setup options* <br>
#@markdown Please check the [latest](https://wiki.blender.org/wiki/Reference/Release_Notes/3.0/Corrective_Releases) version before beginning. [2.83 LTS](https://www.blender.org/download/lts/2-83/) & [2.93 LTS](https://www.blender.org/download/lts/2-89/)

#@markdown ## **Colab Definitions**
blender_version = '3.0.1' #@param ["2.83.19-LTS", "2.93.8-LTS", "3.0.1"] {allow-input: false}
blend_path = 'Blender/projects/'  #@param {type: "string"}
output_path = 'Blender/projects/output/'  #@param {type: "string"}
log_path = 'Blender/logs/log.txt'  #@param {type: "string"}
root_path = '/gdrive/My Drive/'  #@param {type: "string"}
#@markdown ---
#@markdown ## **Blender Preferences**
#@markdown ### Cycles Render Devices
UseGPU = True #@param {type:"boolean"}
UseCPU = True #@param {type:"boolean"}
#@markdown ---
#@markdown ## **Render Properties**
RenderEngine = 'CYCLES' #@param ["CYCLES"] {allow-input: false}
FeatureSet =  'SUPPORTED' #@param ["SUPPORTED", "EXPERIMENTAL"] {allow-input: false}
Device =  'GPU' #@param ["GPU", "CPU"] {allow-input: false}
#@markdown ---
#@markdown ### Sampling
MaxSamples = 400 #@param {type: "integer"}

# Light Paths
# TODO

#@markdown ---
#@markdown ### Performance
#@markdown #### Threads
ThreadsMode = 'AUTO' #@param ["AUTO"] {allow-input: false}
#@markdown #### Memory
UseAutoTile = True #@param {type: "boolean"}
TileSize = '256' #@param [16, 32, 64, 128, 256, 512] {allow-input: false}
#@markdown #### Acceleration Structure
UseSpatialSplits = True #@param {type: "boolean"}
#@markdown #### Final Render
PersistentData = False #@param {type: "boolean"}
#@markdown ---

#@markdown ## **Output Properties**
#@markdown ### Format
# Just basic formats. The others (like Blender's presets) can be added later after after aspect ratio definitions.
Resolution = '1920x1080' #@param ["640x360", "1280x720", "1920x1080", "2560x1440", "3840x2160"] {allow-input: false}
calcRes = Resolution.split('x')
ResolutionX = calcRes[0]
ResolutionY = calcRes[1]

Percentage = 100 #@param {type:"slider", min:25, max:200, step:25}

# Frame Range
# TODO

# Output
# TODO

#@markdown ### Post Processing
Compositing = True #@param {type: "boolean"}

#@markdown ## **Scene Properties**
#@markdown ### Scene
Camera = 'Camera.001' #@param {type: "string"}

#@markdown ---


Download, unpack and move Blender

In [ ]:
blender_mirrors = {'2.83.19-LTS': 'https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.19-linux-x64.tar.xz',
                   '2.93.8-LTS' : 'https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.8-linux-x64.tar.xz',
                   '3.0.1'      : 'https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.0/blender-3.0.1-linux-x64.tar.xz'}

isVerExist = blender_version in blender_mirrors

if isVerExist:
    blender_url = blender_mirrors[blender_version]
    !mkdir $blender_version
    !wget -O '{blender_version}.tar.xz' -nc $blender_url
    !tar xf '{blender_version}.tar.xz' -C ./$blender_version --strip-components=1
else:
    print ("Invalid Blender version. Please check the mirrors.")

This block is required as some weird behaviors with libtcmalloc appeared in the colab VM

In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt update
!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

!echo $LD_PRELOAD

In [ ]:
!apt install libboost-all-dev
!apt install libgl1-mesa-dev
!apt install libglu1-mesa libsm-dev

# Render Config
Required for Blender to use the GPU as expected. In addition you can add custom properties to here.

[Blender CLI Wiki](https://docs.blender.org/manual/en/latest/advanced/command_line/arguments.html)

In [ ]:
#Custom Properties
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = '"+str(Device)+"'\n"+\
    "objects = bpy.data.objects\n"+\
    "collections = bpy.context.view_layer.layer_collection\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "# Attempt to set GPU device types if available\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for scene in bpy.data.scenes:\n"+\
    "    scene.cycles.feature_set = '"+str(FeatureSet)+"'\n"+\
    "    scene.cycles.samples = "+str(MaxSamples)+"\n"+\
    "    scene.render.threads_mode = '"+str(ThreadsMode)+"'\n"+\
    "    scene.cycles.use_auto_tile = "+str(UseAutoTile)+"\n"+\
    "    scene.cycles.tile_size = "+str(TileSize)+"\n"+\
    "    #scene.render.tile_x = "+str(TileSize)+"\n"+\
    "    #scene.render.tile_y = "+str(TileSize)+"\n"+\
    "    scene.cycles.debug_use_spatial_splits = "+str(UseSpatialSplits)+"\n"+\
    "    scene.render.use_persistent_data = "+str(PersistentData)+"\n"+\
    "    scene.render.resolution_x = "+str(ResolutionX)+"\n"+\
    "    scene.render.resolution_y = "+str(ResolutionY)+"\n"+\
    "    scene.render.resolution_percentage = "+str(Percentage)+"\n"+\
    "    #scene.render.use_compositing = "+str(Compositing)+"\n"+\
    "    scene.camera = objects['"+str(Camera)+"']\n"+\
    "\n"+\
    "# bpy.context.objects['Cube'].hide_render = True\n"+\
    "# for collection in collections:\n"+\
    "#    collection['Volume'].hide_render = True\n"+\
    "# bpy.ops.collection.objects_remove(collection='Volume')\n"+\
    "\n"+\
    "# Enable all CPU and GPU devices\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(UseGPU)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(UseCPU)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

#Render animation

Use this if you want to render all Frames

Use `-s` to speficy the start frame.
eg: `-s 10`

Use `-e` to speficy the end frame.
eg: `-e 20`

**THE ORDER IS IMPORTANT. BOTH `-s` AND `-e` MUST BE SPEFICIED BEFORE `-a`**



In [ ]:
!./$blender_version/blender -b -noaudio '{root_path}{blend_path}.blend' -P './setgpu.py' -E {RenderEngine} -o '{root_path}{output_path}' -a |& tee '{root_path}{log_path}'

#Render a frame

Use to render a single frame.
Specify the frame with `-f` *frame_number*

In [ ]:
!./$blender_version/blender -b -noaudio '{root_path}{blend_path}' -P './setgpu.py' -E {RenderEngine} -o '{root_path}{output_path}' -f 1 |& tee '{root_path}{log_path}'